In [229]:
import simpy
import random
from scipy.stats import bernoulli, poisson
from faker import Faker

In [230]:
LUGARES_MAYOR_AFLUENCIA = [('Quito', 0.22), ('Guayaquil', 0.22), ('Cuenca', 0.17), ('Galapagos', 0.15), ('Loja', 0.09), ('Manabi', 0.15)]

CANTIDAD_BUSES = 5
CAPACIDAD_BUS = 40

EDAD_MEDIA_TURISTAS = (25+35)/2
PROBABILIDAD_GENERO_MASCULINO = 0.63 # Mayor probabilidad debido a un índice salarial mejor

TIEMPO_EJECUCION = 7*24*60

SALIDAS_POR_HORA = [x for x in range(60, 30*24*60, 60)]
faker = Faker()

In [231]:
class Persona:
    renegado = None
    def __init__(self, genero=0, edad=-1, tickets=None, origen_destino=list[str]):
        self.genero = genero
        self.nombre = faker.first_name_male() if genero == 1 else faker.first_name_female()
        self.edad = edad
        self.tickets = tickets
        self.origen_destino = origen_destino

class Bus:
    def __init__(self, resource: simpy.Resource, origen_destino, tiempo_viaje):
        self.resource: simpy.Resource = resource
        self.origen_destino = origen_destino
        self.tiempo_viaje = tiempo_viaje
        self.asientos: list[simpy.Resource] = []
        self.estado = 'sin_salir'
        self.proxima_salida = 60
        self.tiempo_regreso = -1

In [232]:
def generar_lugar_origen_destino():
    rst = random.choices(population=list([x[0] for x in LUGARES_MAYOR_AFLUENCIA]), weights=[x[1] for x in LUGARES_MAYOR_AFLUENCIA], k=2)
    while rst[0] == rst[1]:
        rst = random.choices(population=list([x[0] for x in LUGARES_MAYOR_AFLUENCIA]), weights=[x[1] for x in LUGARES_MAYOR_AFLUENCIA], k=2)
    return rst

generar_genero = lambda : bernoulli.rvs(PROBABILIDAD_GENERO_MASCULINO)
generar_edad = lambda : poisson.rvs(mu=EDAD_MEDIA_TURISTAS)

generar_persona = lambda : Persona(generar_genero(), generar_edad())

generar_cantidad_tickets = lambda : random.randint(1, 6)

def generar_tiempo_compra():
    tiempo = random.normalvariate(mu=1, sigma=2) # Media 10 minutos y desviacion de 2 minutos.
    while tiempo<0:
        tiempo = random.normalvariate(mu=1, sigma=2) # Media 10 minutos y desviacion de 2 minutos.
    return tiempo

generar_bus = lambda environment: simpy.Resource(env=environment, capacity=CAPACIDAD_BUS)

def generar_tiempo_viaje():
    tiempo = random.normalvariate(mu=(6*60), sigma=(4*60))
    while tiempo < 0:
        tiempo = random.normalvariate(mu=(6*60), sigma=(4*60))
    return tiempo

In [233]:
ticket_minuto = []
renegado_minuto = []
origen_destino_minuto = []
salidas_minuto = []

class Turismo:
    def __init__(self, environment: simpy.Environment):
        self.environment = environment
        self.buses = [Bus(generar_bus(environment), generar_lugar_origen_destino(), 0) for x in range(5)]

        self.persona: Persona = Persona()
        self.bus = None

    def generar_proceso(self):
        print(print([x.origen_destino for x in self.buses]))
        while True:
            self.persona = Persona(generar_genero(), generar_edad(), generar_cantidad_tickets(), generar_lugar_origen_destino())
            bus: Bus = self.solicitar_bus(self.persona.tickets)
            yield self.environment.process(self.comprar_ticket())

            for bus in self.buses:

                if bus.estado == 'sin_salir' and bus.proxima_salida <= self.environment.now:
                    bus.tiempo_viaje = generar_tiempo_viaje()
                    bus.estado = 'viajando'
                    bus.tiempo_regreso = self.environment.now + bus.tiempo_viaje
                    #print(f'\n \t***** SALIO {self.environment.now} \t ***** REGRESA {bus.tiempo_regreso}***** CANTIDAD PASAJEROS --->{bus.resource.count}<---\n')
                    salidas_minuto.append((1, self.environment.now))
                elif bus.estado == 'viajando' and bus.tiempo_regreso <= self.environment.now:
                    for resource in bus.asientos:
                        bus.resource.release(resource)
                    bus.estado = 'sin_salir'
                    bus.proxima_salida = next(filter(lambda x: x>=bus.tiempo_regreso, SALIDAS_POR_HORA), SALIDAS_POR_HORA[-1]+60)
                    #print(f'\n \t***** REGRESO {self.environment.now}*****')
                    ruta_anterior = bus.origen_destino
                    bus.origen_destino = generar_lugar_origen_destino()
                    #print(f'\t ~~ CAMBIO DE RUTA ~~ RUTA ANTERIOR: {ruta_anterior}, RUTA NUEVA: {bus.origen_destino}\n')

    def solicitar_bus(self, cantidad_tickets):
        bus = list(filter(lambda x: x.origen_destino==self.persona.origen_destino, self.buses))
        if len(bus) > 0:
            self.bus: Bus = bus[0]
            if self.bus.estado == 'sin_salir' and self.bus.resource.count+cantidad_tickets <= self.bus.resource.capacity:
                for i in range(cantidad_tickets):
                    request = self.bus.resource.request()
                    self.bus.asientos.append(request)
                    ticket_minuto.append((cantidad_tickets, self.environment.now))
                    origen_destino_minuto.append((self.bus.origen_destino, self.environment.now))
                return self.bus
        renegado_minuto.append((1, self.environment.now))
        return None

    def comprar_ticket(self):
        tiempo_compra = generar_tiempo_compra()
        yield self.environment.timeout(tiempo_compra)

In [234]:
def simular(environment: simpy.Environment):
    turismo = Turismo(environment)
    yield environment.process(turismo.generar_proceso())

environment = simpy.Environment()
environment.process(simular(environment))
environment.run(until=TIEMPO_EJECUCION)

[['Guayaquil', 'Galapagos'], ['Quito', 'Galapagos'], ['Cuenca', 'Manabi'], ['Cuenca', 'Galapagos'], ['Quito', 'Cuenca']]
None


## REPORTES

In [235]:
from plotly import express
from pandas import DataFrame

dias_en_minutos = [x for x in range(1440, 7*24*60, 60*24)]

## TICKETS VENDIDOS POR DIA

In [236]:
dia_ticket = {x:0 for x in dias_en_minutos}
dia_actual = 0

for dia_en_minuto in dias_en_minutos:
    dia = list(filter(lambda x: x[1]<=dia_en_minuto, ticket_minuto))
    dia_ticket[dia_en_minuto] = sum([x[0] for x in dia])
    temp = [x for x in ticket_minuto if x not in dia]
    ticket_minuto = temp

keys = list(map(lambda x: x/(60*24), list(dia_ticket.keys())))
values = list(dia_ticket.values())

dia_ticket = dict(zip(keys, values))

df = DataFrame.from_dict(dia_ticket.items())
df.columns = ['DIA', 'TICKETS VENDIDOS']
express.bar(df, x='DIA', y='TICKETS VENDIDOS')

## PERSONAS SIN TICKET POR DIA

In [237]:
dia_renegado = {x:0 for x in dias_en_minutos}
dia_actual = 0

for dia_en_minuto in dias_en_minutos:
    dia = list(filter(lambda x: x[1]<=dia_en_minuto, renegado_minuto))
    dia_renegado[dia_en_minuto] = sum([x[0] for x in dia])
    temp = [x for x in renegado_minuto if x not in dia]
    renegado_minuto = temp

keys = list(map(lambda x: x/(60*24), list(dia_renegado.keys())))
values = list(dia_renegado.values())

dia_renegado = dict(zip(keys, values))

df = DataFrame.from_dict(dia_renegado.items())
df.columns = ['DIA', 'RENEGADOS']
express.bar(df, x='DIA', y='RENEGADOS')

## LUGARES VISITADOS POR DIA

In [238]:
from collections import Counter

dia_lugares = {x:0 for x in dias_en_minutos}
dia_actual = 0

for dia_en_minuto in dias_en_minutos:
    dia = list(filter(lambda x:x[1] <= dia_en_minuto, origen_destino_minuto))
    ciudades_visitadas = []
    for valor in dia:
        valor = tuple(list(valor).pop(0))
        ciudades_visitadas.append(valor)
    dia_lugares[dia_en_minuto] = dict(Counter(ciudades_visitadas))


#express.data.medals_long()
df = DataFrame(columns=['dia', 'origen_destino', 'viajes'])
index = 0
for key, values in dia_lugares.items():
    for x, y in values.items():
        df.loc[index] = [key/(60*24)] + [x] + [y]
        index += 1

fig = express.bar(df, x='dia', y='viajes', color='origen_destino', height=1000)
fig.show()

## BUSES POR DiA

In [239]:
dia_salidas = {x:0 for x in dias_en_minutos}
dia_actual = 0

for dia_en_minuto in dias_en_minutos:
    dia = list(filter(lambda x: x[1]<=dia_en_minuto, salidas_minuto))
    dia_salidas[dia_en_minuto] = sum([x[0] for x in dia])
    temp = [x for x in salidas_minuto if x not in dia]
    salidas_minuto = temp

keys = list(map(lambda x: x/(60*24), list(dia_salidas.keys())))
values = list(dia_salidas.values())

dia_salidas = dict(zip(keys, values))

df = DataFrame.from_dict(dia_salidas.items())
df.columns = ['DIA', 'SALIDAS']
express.bar(df, x='DIA', y='SALIDAS')